In [ ]:
import os
import cv2
import glob
import random
import numpy as np

from tensorflow.keras import optimizers
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.applications.inception_resnet_v2 import InceptionResNetV2, preprocess_input
from tensorflow.keras.applications.inception_v3 import InceptionV3, preprocess_input
from tensorflow.keras.applications.mobilenet import MobileNet, preprocess_input
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.applications.vgg19 import VGG19, preprocess_input
from tensorflow.keras.applications.xception import Xception, preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array, array_to_img
from tensorflow.keras.models import load_model, Model, model_from_json, Sequential
from tensorflow.keras.preprocessing import image
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, InputLayer, Dropout, Flatten, GlobalAveragePooling2D
from tensorflow.keras.layers import Dropout, Flatten, GlobalAveragePooling2D


from sklearn.decomposition import PCA, KernelPCA
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import VotingClassifier
from xgboost import XGBClassifier



from load_data_metrics import load_data, metrics, cv_train, results, data_balanced



# Model

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Model
vgg = VGG16(include_top=False, weights='imagenet', input_shape=(224,224,3))
for layer in vgg.layers:
    layer.trainable = False
output = vgg.layers[-1].output
output = Flatten()(output)
vgg_model = Model(vgg.input, output)

# Load dataset

In [ ]:
def load_path(path):
    labels_ = os.listdir(path)
    features = []
    labels   = []
    for i, label in enumerate(labels_):
        cur_path = path + "/" + label 
        for image_path in glob.glob(cur_path + "/*"):
            img = image.load_img(image_path, target_size=(224, 224))
            x = image.img_to_array(img)
            x = np.expand_dims(x, axis=0)
            x = preprocess_input(x)
            feature = vgg_model.predict(x)
            flat = feature.flatten()
            
            features.append(flat)
            labels.append(label)
        print ("[INFO] completed label - " + label)
    return features, labels

data, labels = load_data(load_path)

# Classify

In [ ]:
svm = SVC(C=3, gamma=0.001, kernel='rbf', probability=True)
mlp = MLPClassifier(activation='relu',max_iter=700,alpha=0.001,hidden_layer_sizes=(40,),learning_rate='constant')
xgb = XGBClassifier(colsample_bytree=0.6, max_depth=5, n_estimators=300, subsample= 0.6, seed=0)


models = []
models.append(('SVM', SVC(C=3, gamma=0.001, kernel='rbf')))
models.append(('XGB', XGBClassifier(colsample_bytree=0.6, max_depth=5, n_estimators=300, subsample= 0.6, seed=0))) 
models.append(('MLP', MLPClassifier(activation='relu',max_iter=700,alpha=0.001,hidden_layer_sizes=(40,),learning_rate='constant')))
models.append(('VOT', VotingClassifier(estimators = [('xgb', xgb),('svm', svm),('mlp', mlp)], voting='soft')))


# Results

## Unbalanced

In [ ]:
P0, A0, S0, E0, Fs0, TP0, TN0, FP0, FN0 = cv_train(models, data, labels, 10, 'None', 'vgg')
results(P0, A0, S0, E0, Fs0, TP0, TN0, FP0, FN0)

##  Random Balanced

In [ ]:
P3, A3, S3, E3, Fs3, TP3, TN3, FP3, FN3 = cv_train(models, data, labels, 10, 'random', 'vgg')
results(P3, A3, S3, E3, Fs3, TP3, TN3, FP3, FN3)


## Near Miss Balanced

In [ ]:
P2, A2, S2, E2, Fs2, TP2, TN2, FP2, FN2 = cv_train(models, data, labels, 10, 'nm', 'vgg')
results(P2, A2, S2, E2, Fs2, TP2, TN2, FP2, FN2)

## Cluster Centroid Balanced

In [ ]:
P1, A1, S1, E1, Fs1, TP1, TN1, FP1, FN1 = cv_train(models, data, labels, 10, 'cluster', 'vgg')
results(P1, A1, S1, E1, Fs1, TP1, TN1, FP1, FN1)